In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=37ca12bc2b122cc0551d2aa49d9f021e1960b81286d107e9ec30fbca3f375642
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Spark/Advanced Spark/Most Obscure Superheroes/

/content/drive/My Drive/Spark/Advanced Spark/Most Obscure Superheroes


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType

In [5]:
spark = SparkSession.builder.appName("mostobscuresuperheroes").getOrCreate()

In [6]:
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("name", StringType(), True)])

In [7]:
names = spark.read.schema(schema).option("sep", " ").csv("/content/drive/My Drive/Spark/Advanced Spark/Most Obscure Superheroes/Marvel Names.txt")

In [8]:
lines = spark.read.text("/content/drive/My Drive/Spark/Advanced Spark/Most Obscure Superheroes/Marvel Graph.txt")

In [9]:
connections = lines.withColumn("id", func.split(func.col("value"), " ")[0]).withColumn("connections",func.size(func.split(func.col("value")," "))-1).groupBy("id").agg(func.sum("connections").alias("connections"))

In [10]:
connections.show()

+----+-----------+
|  id|connections|
+----+-----------+
| 691|          7|
|1159|         12|
|3959|        143|
|1572|         36|
|2294|         15|
|1090|          5|
|3606|        172|
|3414|          8|
| 296|         18|
|4821|         17|
|2162|         42|
|1436|         10|
|1512|         12|
|6194|         15|
|6240|         12|
| 829|         38|
|2136|          7|
|5645|         21|
|2069|        264|
| 467|          1|
+----+-----------+
only showing top 20 rows



In [11]:
minConnectionsCount = connections.agg(func.min("connections")).first()[0]

In [12]:
minConnections = connections.filter(func.col("connections") == minConnectionsCount)

In [13]:
minConnectionsNames = minConnections.join(names,"id")

In [14]:
minConnectionsNames.show()

+----+-----------+--------------------+
|  id|connections|                name|
+----+-----------+--------------------+
| 467|          1|        BERSERKER II|
| 577|          1|              BLARE/|
|3490|          1|MARVEL BOY II/MARTIN|
|3489|          1|MARVEL BOY/MARTIN BU|
|2139|          1|      GIURESCU, RADU|
|1089|          1|       CLUMSY FOULUP|
|1841|          1|              FENRIS|
|4517|          1|              RANDAK|
|5028|          1|           SHARKSKIN|
| 835|          1|     CALLAHAN, DANNY|
|1408|          1|         DEATHCHARGE|
|4784|          1|                RUNE|
|4945|          1|         SEA LEOPARD|
|4602|          1|         RED WOLF II|
|6411|          1|              ZANTOR|
|3014|          1|JOHNSON, LYNDON BAIN|
|3298|          1|          LUNATIK II|
|2911|          1|                KULL|
|2117|          1|GERVASE, LADY ALYSSA|
+----+-----------+--------------------+

